titulos # - subtitulos  ##

# ***<p style="text-align:center;">Análise de Texto através de Bases de Dados Vetoriais</p>***

## Resumo

Este relatório representa o processo de desenvolvimento e implementação de uma Base de Dados Vetorial com o objetivo de serem identificados padrões de texto.

Como Base de Dados vetorial decidimos utilizar **ChromaDB** devido à sua excelente integração com Python aliada com a sua facilidade de configuração.  
O Projeto dividiu-se essencialmente em 3 fases. **Aquição e Processamento de Dados**, **Geração de Embeddings e Armazenamento Vetorial** e **Pesquisa por Similaridade**. Em seguida abordamos cada uma detalhadamente.

Atualmente, é possível carregar variados tipos de ficheiros e pesquisar por um termo ou texto sendo obtido os ficheiros com o conteúdo mais similar.

## Introdução

Como mencionamos anteriormente o nosso projeto foi essencialmente divido em 3 fases.

Relativamente à Aquisição e Processamendo de Dados, a nossa aplicação está preparada para ler ficheiros de texto de 3 tipos: PDF, TXT e DOCX. O conteúdo destes ficheiros é lido e guardado num dicionário onde a chave é o nome do ficheiro e o valor o conteúdo do mesmo.

Quanto à Geração de Embeddings e Armazenamento Vetorial, usamos o modelo default do ChromaDB *"all-MiniLM-L6-v2"* que irá converter o texto para um vetor e armazená-lo na Base de Dados.

Por fim, é efetuada a procura por similaridade. O Modelo converte a query (input do utilizador) e compara com os vetores previamente armazenados. Neste caso, o modelo escolhido por nós usa a Distância Euclidiana para calcular, onde a proximidade entre vetores representa a similaridade semântica. De seguida, retorna o número de resultados requisitado.

### importação biblioteca para usar o sistemas de base de dados Chromadb

In [1]:
import chromadb
from chromadb.utils import embedding_functions

importação de funcionalidades cridas fazer a leitura de arquivos a ser usados durante este projeto

In [2]:
import leitor

Importação de bibliotecas necessárias para obter a data e hora com o objetivo fututo de realizar o "benchmarking" dos diversos modelos de conversão de embeddings  

In [3]:
import os

Criar o cliente para o ChromaDB

In [4]:
chroma_client = chromadb.Client(
    chromadb.config.Settings(chroma_server_host="chroma", chroma_server_http_port="8000")
)

Escolher o modelo de embeddings, para a conversão do material fornecido pelo Utizador nos respetivos ficheiros "data" em embeddings vetoriais

In [5]:
embedder = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

c:\Users\José Pedro\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Neste ponto podemos criar a coleção de embeddings numa estrutura de dados predefinida do ChromaDB.

Uma coleção no ChromaDB é um agrupamento com nome de embeddings, dos seus documentos correspondentes e dos respetivos metadados, onde ocorre o processo de organização, gerir e, de forma crucial, efetuar pesquisas por similaridade em dados vetorizados

No caso do ChromaDB, uma coleção é o local fundamental onde se armazenam e organizam os embeddings vetoriais, os dados originais que estes representam, no caso do nosso projeto trabalho com dados do tipo .doc, .pdf e .txt, e os metadados associados.

Como é referido anteriormente, também é possivel trabalhar com outros tipo de dados, tendo o exemplo das imagens,

Assim sendo, na criação da coleção, é necessário definir o nome da coleção  ( name="my_collection" ), o modelo de embeddings ( embedding_function=embedder, sendo o embedder definido pelo Utilizador anteriormente) a utilizar e os metadados que se pretende associar a cada embedding.



In [6]:
collection = chroma_client.get_or_create_collection(
        name="my_collection",
        embedding_function=embedder, 
)

Função para adicionar os dados lidos dos ficheiros à coleção.

In [7]:
def adiciona_dados(collection, dados):
    # Adicionar ficheiros à coleção
    collection.add(
        documents=list(dados.values()),  # lista de ficheiros
        ids=[k for k in dados.keys()]    # ids dos ficheiros
    )
    print(f"Foram adicionados {len(dados)} ficheiros.")
    # Limpar o dicionário para evitar adicionar os mesmos ficheiros novamente
    dados.clear()

Após a criação da coleção, criamos um dicionário vazio para armazenar os metadados associados a cada embedding, onde o nome do ficheiro é a chave e o conteúdo do ficheiro é o valor.

In [8]:
dados = {}

Ler ficheiros pdf

In [9]:
leitor.extract_text(r"..\data/pdf", dados)
adiciona_dados(collection, dados)

Foram adicionados 10 ficheiros.


Ler ficheiros docx

In [10]:
leitor.extract_text(r"..\data/txt", dados)
adiciona_dados(collection, dados)

Foram adicionados 10 ficheiros.


Ler ficheiros txt

In [11]:
leitor.extract_text(r"..\data/docx", dados)
adiciona_dados(collection, dados)

Foram adicionados 10 ficheiros.


Para efetuar a pesquisa por similaridade, o utilizador escreve o termo ou texto que deseja pesquisar e o número de resultados que deseja obter. Através da função query o termo ou texto é convertido para um embedding e comparado com os embeddings armazenados na coleção, armazenando os resultados mais semelhantes na variável "results".

In [12]:
prompt = input("Digite o termo que deseja pesquisar: ")
n_resultados = int(input("Quantos resultados deseja obter:  "))
results = collection.query(
        query_texts=[prompt],    # o que o utilizador quer pesquisar                  
        n_results=n_resultados   # Número de resultados a serem retornados
)

Após serem obtidos os resultados da pesquisa, processamos os resultados obtidos, extraindo os textos dos ficheiros correspondentes aos embeddings mais semelhantes encontrados na coleção assim como o nome do ficheiro e a distância entre o embedding de consulta e os embeddings encontrados na coleção.

In [13]:
num_results = len(results['documents'][0])
doc_text = []
titles = []
distance = []
for i in range(num_results):
    doc_text.append(results['documents'][0][i])
    distance.append(results['distances'][0][i])
    doc_id_path = results['ids'][0][i]
    titles.append(os.path.basename(doc_id_path))


Apresentamos ao utilizador os títulos dos ficheiros correspondentes aos embeddings mais semelhantes encontrados na coleção, juntamente com a distância entre o embedding de consulta e os embeddings encontrados na coleção. A distância é uma medida que indica o quão semelhante é o embedding de consulta em relação aos embeddings encontrados na coleção.

In [14]:
print(f"\n--- Pesquisas encontradas ---\n")   
j = 0 
for title in titles:
    print(f"--- Resultado {j+1} ---")
    print(f"Ficheiro: {title}")
    print(f"Distância de Similaridade: {distance[j]:.4f}\n")
    j += 1



--- Pesquisas encontradas ---

--- Resultado 1 ---
Ficheiro: Cinderela.pdf
Distância de Similaridade: 1.5345

--- Resultado 2 ---
Ficheiro: O-Soldadinho-de-Chumbo.pdf
Distância de Similaridade: 1.5567

--- Resultado 3 ---
Ficheiro: O talisman.docx
Distância de Similaridade: 1.6004



O utilizador pode então selecionar o resultado que deseja visualizar, sendo exibido o conteúdo do ficheiro. 

In [15]:

choice = input("Digite o número do resultado desejado: ")
try:
    choice = int(choice) - 1  # Ajustar para índice 0
    if 0 <= choice < num_results:
        print("============================================\n")
        print(f"Conteúdo do Resultado {choice + 1}:")
        print(f"Ficheiro: {titles[choice]}")
        print(f"{doc_text[choice]}\n\n")
        
    else:
        print("Opção inválida!")
except ValueError:
    print("Por favor, escreva apenas números.")




Conteúdo do Resultado 1:
Ficheiro: Cinderela.pdf
Há muito tempo, houve uma vez uma linda e doce menina. Quando ela era 
muito pequena, sua mãe ficou doente. No leito de morte, a mãe chamou sua 
filha para ela. Ela disse: “Sempre viva bem e honestamente e o mundo 
também será bom para você.” Logo após essas palavras, ela fechou seus 
olhos para sempre. O pai da menina era um rico homem de negócios que 
viajava muito. Pouco tempo depois da morte de sua esposa, ele se casou 
novamente.  
 
Com sua nova esposa, o empresário teve mais duas filhas. Eram lindas por 
fora, mas horríveis por dentro e eram muito más com sua meia -irmã. 
Levaram suas belas roupas e fizeram -na fazer todas as tarefas da casa. À 
noite, ela não tinha permissão para dormir em uma cama macia, mas tinha 
que deitar na frente da lareira. Como resultado , suas roupas estavam 
sempre cobertas de cinzas e ela foi chamada de Cinderela. Cinderela 
pensava constantemente nas palavras de sua mãe e nunca reclamou de nada 
que

Para finalizar, também é possível eliminar dados da coleção, é apresentada ao utilizador uma lista com os ids (path do ficheiro) dos embeddings que pertencem à coleção. O utilizador pode escolher o id do embedding que deseja eliminar, sendo este removido da coleção.

In [16]:
ids = collection.get(include=[])
for id in ids['ids']:
    print(id)
id = input("Digite o ID do ficheiro que deseja eliminar: ")
if id in ids['ids']:
    collection.delete(ids=[id])
    print(f"Ficheiro {id} eliminado com sucesso.")
else:
    print(f"Ficheiro {id} não encontrado.")

..\data/pdf\A-rena-que-não-sabia-voar.pdf
..\data/pdf\Bart_-O-Urso-vai-Hibernar.pdf
..\data/pdf\Cinderela.pdf
..\data/pdf\Medo-do-cabeleireiro.pdf
..\data/pdf\O-Elfo-Travesso-E-O-Homem-Biscoito.pdf
..\data/pdf\o-menino.pdf
..\data/pdf\O-Soldadinho-de-Chumbo.pdf
..\data/pdf\Rumpelstilzinho.pdf
..\data/pdf\sodapdf-converted-1.pdf
..\data/pdf\sodapdf-converted.pdf
..\data/txt\A lupa e o binóculo.txt
..\data/txt\Andar à caranguejo.txt
..\data/txt\Conselhos de mocho não chegam ao chão.txt
..\data/txt\Duas gaivotas.txt
..\data/txt\Ele há cada nome….txt
..\data/txt\Não havendo energia….txt
..\data/txt\O pau de fósforo.txt
..\data/txt\Quem caça quer caça.txt
..\data/txt\Tem horas.txt
..\data/txt\Zé desgraçado.txt
..\data/docx\A alma.docx
..\data/docx\A canção da cerejeira.docx
..\data/docx\Alberto.docx
..\data/docx\Doçura e bondade.docx
..\data/docx\Não quero.docx
..\data/docx\O ouro.docx
..\data/docx\O pinheiro ambicioso.docx
..\data/docx\O rico e o pobre.docx
..\data/docx\O talisman.docx
..\

Dentro da pasta app, existe um ficheiro chamado main.py, que é o ponto de entrada da aplicação. Este ficheiro contém o código principal que executa a aplicação e inicia um menu interativo para o utilizador. O menu permite ao utilizador escolher entre diferentes opções, como adicionar novos embeddings à coleção, realizar pesquisas por similaridade ou limpar a coleção existente.